### Data Ingetion

In [39]:
### Document Structure

from langchain_core.documents import Document

In [40]:
doc=Document(
    page_content="This is the main text content i am using to create RAG",
    metadata={
        "source": "example.txt",
        "pages": 1,
        "author": "Yashraj Ghosalkar",
        "date_creted": "2025-01-01"
    }
)
doc

Document(metadata={'source': 'example.txt', 'pages': 1, 'author': 'Yashraj Ghosalkar', 'date_creted': '2025-01-01'}, page_content='This is the main text content i am using to create RAG')

In [41]:
## create a simple txt file
import os
os.makedirs("../data/text_files",exist_ok=True)

In [42]:
sample_texts={
    "../data/text_files/python_intro.txt":"""Python Programming Introduction

Python is a high-level, interpreted programming language known for its simplicity and readability.
Created by Guido van Rossum and first released in 1991, Python has become one of the most popular
programming languages in the world.

Key Features:
- Easy to learn and use
- Extensive standard library
- Cross-platform compatibility
- Strong community support

Python is widely used in web development, data science, artificial intelligence, and automation.""",
    
    "../data/text_files/machine_learning.txt": """Machine Learning Basics

Machine learning is a subset of artificial intelligence that enables systems to learn and improve
from experience without being explicitly programmed. It focuses on developing computer programs
that can access data and use it to learn for themselves.

Types of Machine Learning:
1. Supervised Learning: Learning with labeled data
2. Unsupervised Learning: Finding patterns in unlabeled data
3. Reinforcement Learning: Learning through rewards and penalties

Applications include image recognition, speech processing, and recommendation systems
    
    
    """

}

for filepath,content in sample_texts.items():
    with open(filepath,'w',encoding="utf-8") as f:
        f.write(content)

print("✅ Sample text files created!")

✅ Sample text files created!


In [43]:
### TextLoader
# from langchain.document_loaders import TextLoader // was the old way

from langchain_community.document_loaders import TextLoader

loader=TextLoader("../data/text_files/python_intro.txt",encoding="utf-8")
document=loader.load()
print(document)


[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.')]


In [44]:
### Directory Loader
from langchain_community.document_loaders import DirectoryLoader

## load all the text files from the directory
dir_loader=DirectoryLoader(
    "../data/text_files",
    glob="**/*.txt",
    loader_cls= TextLoader,
    loader_kwargs={'encoding': 'utf-8'},
    show_progress=False
)

documents=dir_loader.load()
documents

[Document(metadata={'source': '../data/text_files/python_intro.txt'}, page_content='Python Programming Introduction\n\nPython is a high-level, interpreted programming language known for its simplicity and readability.\nCreated by Guido van Rossum and first released in 1991, Python has become one of the most popular\nprogramming languages in the world.\n\nKey Features:\n- Easy to learn and use\n- Extensive standard library\n- Cross-platform compatibility\n- Strong community support\n\nPython is widely used in web development, data science, artificial intelligence, and automation.'),
 Document(metadata={'source': '../data/text_files/machine_learning.txt'}, page_content='Machine Learning Basics\n\nMachine learning is a subset of artificial intelligence that enables systems to learn and improve\nfrom experience without being explicitly programmed. It focuses on developing computer programs\nthat can access data and use it to learn for themselves.\n\nTypes of Machine Learning:\n1. Supervise

In [45]:
## for PDFs
from langchain_community.document_loaders import PyMuPDFLoader

## load all the text files from the directory
dir_loader=DirectoryLoader(
    "../data/pdf",
    glob="**/*.pdf",
    loader_cls= PyMuPDFLoader,
    show_progress=False
)

pdf_documents=dir_loader.load()
pdf_documents

[Document(metadata={'producer': 'pdfTeX-1.40.27', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-12-15T08:03:20+00:00', 'source': '../data/pdf/Yashraj_Resume_Nov_2025.pdf', 'file_path': '../data/pdf/Yashraj_Resume_Nov_2025.pdf', 'total_pages': 1, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2025-12-15T08:03:20+00:00', 'trapped': '', 'modDate': 'D:20251215080320Z', 'creationDate': 'D:20251215080320Z', 'page': 0}, page_content='Yashraj Dagadu Ghosalkar\nEmail: yashrajjghosalkar@gmail.com\nLinkedIn | GitHub | mmrstatistics.com\nMobile: +91-8010881394\nSummary\nFull-stack engineer who built and operates a live SaaS (MMR Statistics) end-to-end—architecture →payments →\nSEO—on Next.js/TypeScript/AWS. Sole web dev at MMR; led 4 interns, ship production features fast, and keep\nquality high.\nSkills\n• Programming: JavaScript (ES6+), TypeScript, Python, PHP (WordPress), SQL, HTML5, CSS3\n• Frontend: React, Next.js (App Router, RSC), Redux,

In [46]:
## MongoDB Loader
# DATABASE_URL=

# add this import for running in jupyter notebook
import nest_asyncio

nest_asyncio.apply()

from langchain_community.document_loaders.mongodb import MongodbLoader

db_loader = MongodbLoader(
    connection_string="",
    db_name="mmr-statistics",
    collection_name="statistics",
    field_names=["title","region","canonicalUrl","description"]
)

docs = db_loader.load()

len(docs)

docs[0]


ValueError: connection_string must be provided.

In [ ]:
import nest_asyncio
from langchain_community.document_loaders.mongodb import MongodbLoader
from langchain_core.documents import Document

nest_asyncio.apply()

# 1. Helper function to extract text from your specific JSON structure
def extract_text_from_prosemirror(node):
    """Recursively extracts text from the Tiptap/ProseMirror JSON structure."""
    if isinstance(node, list):
        return " ".join([extract_text_from_prosemirror(i) for i in node])
    if isinstance(node, dict):
        text = ""
        if node.get("type") == "text":
            text += node.get("text", "")
        if "content" in node:
            text += extract_text_from_prosemirror(node["content"])
        return text
    return ""

# 2. Load the raw documents
loader = MongodbLoader(
    connection_string="",
    db_name="mmr-statistics",
    collection_name="statistics",
    field_names=["title", "region", "canonicalUrl", "description"]
)

raw_docs = loader.load()

# 3. Process the documents to clean the 'description'
cleaned_docs = []

for doc in raw_docs:
    # The MongodbLoader puts everything in page_content as a string.
    # To handle this properly, we actually need the raw dictionary.
    # If MongodbLoader isn't giving you the dict, we can fetch it manually:
    
    # Extract the 'description' if it's stored in metadata (it usually isn't in the default loader)
    # Since the default loader flattens everything into a string, 
    # a cleaner way is to use the motor client directly or use this 'Hack':
    
    # Note: If your description is already a string representation of a dict in page_content,
    # we convert the logic to handle the raw data before it becomes a 'Document'.
    pass

### RECOMMENDED: The "Clean" Way using Motor/PyMongo
# Because LangChain's MongodbLoader is limited for complex JSON, 
# it's often better to fetch the data and then create LangChain Documents manually.

from pymongo import MongoClient

client = MongoClient("")
collection = client["mmr-statistics"]["statistics"]

# Fetch records
cursor = collection.find({}, {"title": 1, "region": 1, "canonicalUrl": 1, "description": 1})

final_langchain_docs = []

for item in cursor:
    # Extract clean text from the description JSON
    description_text = extract_text_from_prosemirror(item.get("description", {}))
    
    # Construct the content string you want
    content = f"Title: {item.get('title')}\nRegion: {item.get('region')}\nURL: {item.get('canonicalUrl')}\nDescription: {description_text}"
    
    # Create the Document object
    new_doc = Document(
        page_content=content,
        metadata={"source": item.get("canonicalUrl"), "id": str(item.get("_id"))}
    )
    final_langchain_docs.append(new_doc)

# Now check the result
print(final_langchain_docs[0].page_content)

Title: India: Gross domestic product (GDP) in current prices from 1987 to 2030
Region: india
URL: http://localhost:3000/statistics/695388/india-gross-domestic-product-gdp-in-current-prices-from-1987-to-2030
Description: The statistic shows GDP in India from 1987 to 2024, with projections up until 2030. In 2024, GDP in  India  was at around 3.91 trillion U.S. dollars, and it is expected to reach six trillion by the end of the decade. See figures on  India's economic growth here , and the  Russian GDP  for comparison.    Historical development of the Indian economy In the 1950s and 1960s, the decision of the newly independent Indian government to adopt a mixed economy, adopting both elements of both capitalist and socialist systems, resulted in huge inefficiencies borne out of the culture of interventionism that was a direct result of the lackluster implementation of policy and failings within the system itself. The desire to move towards a Soviet style mass planning system failed to gai

In [ ]:
import nest_asyncio
from pymongo import MongoClient
from langchain_core.documents import Document

nest_asyncio.apply()

# 1. Recursive Helper to clean any JSON content (Description, Strategic Insights, etc.)
def extract_text_from_json(node):
    if not node: return ""
    if isinstance(node, str): return node
    if isinstance(node, list):
        return " ".join([extract_text_from_json(i) for i in node])
    if isinstance(node, dict):
        text = ""
        if node.get("type") == "text":
            text += node.get("text", "")
        # Recursively check 'content' or other nested structures
        for key in ["content", "customSectionDescription"]:
            if key in node:
                text += " " + extract_text_from_json(node[key])
        return text.strip()
    return ""

# 2. Database Connection
MONGO_URI = ""
client = MongoClient(MONGO_URI)
db = client["mmr-statistics"]

# 3. The Aggregation Pipeline (Joining Collections)
pipeline = [
    {
        "$lookup": {
            "from": "topics", # Collection name for Topics
            "localField": "topicId",
            "foreignField": "_id",
            "as": "topic_info"
        }
    },
    {
        "$lookup": {
            "from": "markets", # Collection name for Industry/Markets
            "localField": "industryId",
            "foreignField": "_id",
            "as": "market_info"
        }
    },
    {
        "$lookup": {
            "from": "users", # Collection name for Authors
            "localField": "authorId",
            "foreignField": "_id",
            "as": "author_info"
        }
    },
    {
        "$lookup": {
            "from": "charts",
            "localField": "chartId",
            "foreignField": "_id",
            "as": "chart_info"
        }
    },
    # Unwind arrays to objects (if they exist)
    {"$unwind": {"path": "$topic_info", "preserveNullAndEmptyArrays": True}},
    {"$unwind": {"path": "$market_info", "preserveNullAndEmptyArrays": True}},
    {"$unwind": {"path": "$author_info", "preserveNullAndEmptyArrays": True}},
    {"$unwind": {"path": "$chart_info", "preserveNullAndEmptyArrays": True}}
]

raw_data = list(db["statistics"].aggregate(pipeline))

# 4. Convert to LangChain Documents
langchain_docs = []

for item in raw_data:
    # --- Clean the JSON fields ---
    desc = extract_text_from_json(item.get("description"))
    insights = extract_text_from_json(item.get("strategicInsights"))
    
    # --- Gather Joined Info ---
    topic_name = item.get("topic_info", {}).get("name", "N/A")
    market_name = item.get("market_info", {}).get("name", "N/A")
    author_name = f"{item.get('author_info', {}).get('firstName', '')} {item.get('author_info', {}).get('lastName', '')}".strip() or "Unknown"
    
    # --- Process Chart Data (Optional: Convert rows to text) ---
    chart_text = ""
    if "chart_info" in item:
        chart_data = item["chart_info"].get("chartData", [])
        chart_text = " | ".join([f"{d.get('Category')}: {d.get('Value')}" for d in chart_data])

    # --- Construct Page Content ---
    page_content = f"""
TITLE: {item.get('title')}
MARKET/INDUSTRY: {market_name}
TOPIC: {topic_name}
AUTHOR: {author_name}
REGION: {item.get('region')}
URL: {item.get('canonicalUrl')}

DESCRIPTION:
{desc}

STRATEGIC INSIGHTS:
{insights}

CHART DATA:
{chart_text}
    """.strip()

    # --- Create Document with Metadata ---
    doc = Document(
        page_content=page_content,
        metadata={
            "source": item.get("canonicalUrl"),
            "statistic_id": str(item.get("_id")),
            "market": market_name,
            "topic": topic_name,
            "author": author_name
        }
    )
    langchain_docs.append(doc)

# 5. Verify first doc
if langchain_docs:
    print(f"Loaded {len(langchain_docs)} documents.")
    print("--- FIRST DOCUMENT CONTENT ---")
    print(langchain_docs[0].page_content[:100000] + "...")


Loaded 10 documents.
--- FIRST DOCUMENT CONTENT ---
TITLE: India: Gross domestic product (GDP) in current prices from 1987 to 2030
MARKET/INDUSTRY: Consumer Goods and Services
TOPIC: Skin Enhancers
AUTHOR: Admin User
REGION: india
URL: http://localhost:3000/statistics/695388/india-gross-domestic-product-gdp-in-current-prices-from-1987-to-2030

DESCRIPTION:
The statistic shows GDP in India from 1987 to 2024, with projections up until 2030. In 2024, GDP in India was at around 3.91 trillion U.S. dollars, and it is expected to reach six trillion by the end of the decade. See figures on India's economic growth here , and the Russian GDP for comparison.   Historical development of the Indian economy In the 1950s and 1960s, the decision of the newly independent Indian government to adopt a mixed economy, adopting both elements of both capitalist and socialist systems, resulted in huge inefficiencies borne out of the culture of interventionism that was a direct result of the lackluster impleme

In [ ]:



import nest_asyncio
from pymongo import MongoClient
from langchain_core.documents import Document

nest_asyncio.apply()

# --- HELPER: Convert Chart rawData to Markdown Table ---
def format_chart_to_markdown(raw_data_obj):
    if not raw_data_obj or "headers" not in raw_data_obj:
        return "No chart data available."
    
    headers = raw_data_obj.get("headers", [])
    rows = raw_data_obj.get("rows", [])
    
    # Create Markdown Header
    md_table = "| " + " | ".join(headers) + " |\n"
    md_table += "| " + " | ".join(["---"] * len(headers)) + " |\n"
    
    # Add Rows
    for row in rows:
        md_table += "| " + " | ".join(map(str, row)) + " |\n"
    
    return md_table

# --- HELPER: Recursive JSON Content Extractor ---
def extract_text_from_json(node):
    if not node: return ""
    if isinstance(node, str): return node
    if isinstance(node, list):
        return " ".join([extract_text_from_json(i) for i in node])
    if isinstance(node, dict):
        text = ""
        if node.get("type") == "text":
            text += node.get("text", "")
        # Check standard Tiptap fields
        for key in ["content", "customSectionDescription"]:
            if key in node:
                text += " " + extract_text_from_json(node[key])
        return text.strip()
    return ""

MONGODB_URI = os.getenv("MONGODB_URI")
MONGODB_DB_NAME = os.getenv("MONGODB_DB_NAME")

if not MONGODB_URI or not MONGODB_DB_NAME:
    raise ValueError("MongoDB environment variables are not set")

client = MongoClient(MONGODB_URI)
db = client[MONGODB_DB_NAME]


pipeline = [
    {"$lookup": {"from": "topics", "localField": "topicId", "foreignField": "_id", "as": "topic_info"}},
    {"$lookup": {"from": "markets", "localField": "industryId", "foreignField": "_id", "as": "market_info"}},
    {"$lookup": {"from": "users", "localField": "authorId", "foreignField": "_id", "as": "author_info"}},
    {"$lookup": {"from": "charts", "localField": "chartId", "foreignField": "_id", "as": "chart_info"}},
    {"$unwind": {"path": "$topic_info", "preserveNullAndEmptyArrays": True}},
    {"$unwind": {"path": "$market_info", "preserveNullAndEmptyArrays": True}},
    {"$unwind": {"path": "$author_info", "preserveNullAndEmptyArrays": True}},
    {"$unwind": {"path": "$chart_info", "preserveNullAndEmptyArrays": True}}
]

raw_data = list(db["statistics"].aggregate(pipeline))
langchain_docs = []

for item in raw_data:
    # 1. Parse Text Content
    desc = extract_text_from_json(item.get("description"))
    insights = extract_text_from_json(item.get("strategicInsights"))
    
    # 2. Extract Names
    topic_name = item.get("topic_info", {}).get("name", "N/A")
    market_name = item.get("market_info", {}).get("name", "N/A")
    author_name = f"{item.get('author_info', {}).get('firstName', '')} {item.get('author_info', {}).get('lastName', '')}".strip()
    
    # 3. Process Chart Data & Metadata
    chart_info = item.get("chart_info", {})
    chart_table = format_chart_to_markdown(chart_info.get("rawData", {}))
    
    # Extract Axis labels and Units for better context
    config = chart_info.get("chartConfig", {})
    x_label = config.get("xLabel", "Category")
    y_label = config.get("primaryYAxis", {}).get("label", "Value")
    unit = config.get("primaryYAxis", {}).get("unit", item.get("unit", "N/A"))

    # 4. Construct Final Structured Content
    page_content = f"""
# {item.get('title')}

## Categorization
- **Market/Industry:** {market_name}
- **Topic:** {topic_name}
- **Region:** {item.get('region')}
- **Source:** {item.get('sourceName')} ({author_name})

## Summary & Description
{desc}

## Strategic Insights
{insights}

## Data Table ({unit})
*X-Axis: {x_label} | Y-Axis: {y_label}*

{chart_table}

---
*Reference: {item.get('canonicalUrl')}*
    """.strip()

    # 5. Build Document
    doc = Document(
        page_content=page_content,
        metadata={
            "id": str(item.get("_id")),
            "market": market_name,
            "topic": topic_name,
            "author": author_name,
            "region": item.get('region'),
            "has_chart": bool(chart_info)
        }
    )
    langchain_docs.append(doc)

print(f"Total documents prepared: {len(langchain_docs)}")


Total documents prepared: 10


In [ ]:
print("\n--- SAMPLE DOCUMENT PREVIEW ---")
print(langchain_docs[0].page_content)



--- SAMPLE DOCUMENT PREVIEW ---
# India: Gross domestic product (GDP) in current prices from 1987 to 2030

## Categorization
- **Market/Industry:** Consumer Goods and Services
- **Topic:** Skin Enhancers
- **Region:** india
- **Source:** Maximize Market Research (Admin User)

## Summary & Description
The statistic shows GDP in India from 1987 to 2024, with projections up until 2030. In 2024, GDP in India was at around 3.91 trillion U.S. dollars, and it is expected to reach six trillion by the end of the decade. See figures on India's economic growth here , and the Russian GDP for comparison.   Historical development of the Indian economy In the 1950s and 1960s, the decision of the newly independent Indian government to adopt a mixed economy, adopting both elements of both capitalist and socialist systems, resulted in huge inefficiencies borne out of the culture of interventionism that was a direct result of the lackluster implementation of policy and failings within the system itself.

# Embedding And vectorStoreDB

In [ ]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity


In [ ]:
class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-V2"):
        """
        Initialise the emvedding manager
        
        args:
        model_name: HuggingFace model name for sentence embeddings    
        """
        self.model_name = model_name
        self.model = None
        self._load_model()
        
    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise
        
    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-V2


Loading weights: 100%|██████████| 103/103 [00:00<00:00, 1403.44it/s, Materializing param=pooler.dense.weight]                             
BertModel LOAD REPORT from: sentence-transformers/all-MiniLM-L6-V2
Key                     | Status     |  | 
------------------------+------------+--+-
embeddings.position_ids | UNEXPECTED |  | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.


Model loaded successfully. Embedding dimension: 384


# VectorStore

In [ ]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas, 
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore
    

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 0


In [ ]:
import os
from pathlib import Path
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [ ]:
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data")

Found 3 PDF files to process

Processing: Yashraj_Resume_Nov_2025.pdf
  ✓ Loaded 1 pages

Processing: Yashraj_Ghosalkar_Jan.pdf
  ✓ Loaded 1 pages

Processing: Yashraj_MMR_Anniversary_Speech_Final.pdf
  ✓ Loaded 4 pages

Total documents loaded: 6


In [ ]:
all_pdf_documents


[Document(metadata={'producer': 'pdfTeX-1.40.27', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-12-15T08:03:20+00:00', 'author': '', 'keywords': '', 'moddate': '2025-12-15T08:03:20+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.27 (TeX Live 2025) kpathsea version 6.4.1', 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/pdf/Yashraj_Resume_Nov_2025.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Yashraj_Resume_Nov_2025.pdf', 'file_type': 'pdf'}, page_content='Yashraj Dagadu GhosalkarEmail: yashrajjghosalkar@gmail.com\nLinkedIn|GitHub|mmrstatistics.com Mobile: +91-8010881394\nSummary\nFull-stack engineer who built and operates a live SaaS (MMR Statistics) end-to-end—architecture→payments→\nSEO—on Next.js/TypeScript/A WS. Sole web dev at MMR; led 4 interns, ship production features fast, and keep\nquality high.\nSkills\n•Programming: JavaScript (ES6+), TypeScript, Python, PHP (WordPress), SQL, HTML5, CSS3\n•F ront

In [ ]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs


In [ ]:
chunks=split_documents(all_pdf_documents)
chunks

Split 6 documents into 17 chunks

Example chunk:
Content: Yashraj Dagadu GhosalkarEmail: yashrajjghosalkar@gmail.com
LinkedIn|GitHub|mmrstatistics.com Mobile: +91-8010881394
Summary
Full-stack engineer who built and operates a live SaaS (MMR Statistics) end-...
Metadata: {'producer': 'pdfTeX-1.40.27', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-12-15T08:03:20+00:00', 'author': '', 'keywords': '', 'moddate': '2025-12-15T08:03:20+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.27 (TeX Live 2025) kpathsea version 6.4.1', 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/pdf/Yashraj_Resume_Nov_2025.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Yashraj_Resume_Nov_2025.pdf', 'file_type': 'pdf'}


[Document(metadata={'producer': 'pdfTeX-1.40.27', 'creator': 'LaTeX with hyperref', 'creationdate': '2025-12-15T08:03:20+00:00', 'author': '', 'keywords': '', 'moddate': '2025-12-15T08:03:20+00:00', 'ptex.fullbanner': 'This is pdfTeX, Version 3.141592653-2.6-1.40.27 (TeX Live 2025) kpathsea version 6.4.1', 'subject': '', 'title': '', 'trapped': '/False', 'source': '../data/pdf/Yashraj_Resume_Nov_2025.pdf', 'total_pages': 1, 'page': 0, 'page_label': '1', 'source_file': 'Yashraj_Resume_Nov_2025.pdf', 'file_type': 'pdf'}, page_content='Yashraj Dagadu GhosalkarEmail: yashrajjghosalkar@gmail.com\nLinkedIn|GitHub|mmrstatistics.com Mobile: +91-8010881394\nSummary\nFull-stack engineer who built and operates a live SaaS (MMR Statistics) end-to-end—architecture→payments→\nSEO—on Next.js/TypeScript/A WS. Sole web dev at MMR; led 4 interns, ship production features fast, and keep\nquality high.\nSkills\n•Programming: JavaScript (ES6+), TypeScript, Python, PHP (WordPress), SQL, HTML5, CSS3\n•F ront

In [ ]:
### convert the text to emveddings
texts=[doc.page_content for doc in chunks]
texts   

['Yashraj Dagadu GhosalkarEmail: yashrajjghosalkar@gmail.com\nLinkedIn|GitHub|mmrstatistics.com Mobile: +91-8010881394\nSummary\nFull-stack engineer who built and operates a live SaaS (MMR Statistics) end-to-end—architecture→payments→\nSEO—on Next.js/TypeScript/A WS. Sole web dev at MMR; led 4 interns, ship production features fast, and keep\nquality high.\nSkills\n•Programming: JavaScript (ES6+), TypeScript, Python, PHP (WordPress), SQL, HTML5, CSS3\n•F rontend: React, Next.js (App Router, RSC), Redux, Tailwind CSS, shadcn/ui\n•Backend & APIs: Node.js, Express.js, REST, Webhooks, JWT/Auth, RBAC, Cron Jobs\n•Auth & Access: NextAuth (Google/OAuth & credentials), role/permission middleware, route protection\n•Payments: HDFC SmartGateway/Juspay flows, checkout orchestration, order lifecycles\n•Data: MongoDB (Mongoose models, indexes), MySQL\n•Cloud/DevOps: AWS (EC2, S3, CloudFront), Docker, CI/CD (GitHub Actions), Linux, Cloudflare Turnstile',
 '•Data: MongoDB (Mongoose models, indexes), 

In [ ]:
### convert the text to emveddings
texts=[doc.page_content for doc in chunks]
texts   

## Generate the Emveddings

embeddings=embedding_manager.generate_embeddings(texts)


## store in the vector database
vectorstore.add_documents(chunks, embeddings)

Generating embeddings for 17 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00,  2.53it/s]

Generated embeddings with shape: (17, 384)
Adding 17 documents to vector store...
Successfully added 17 documents to vector store
Total documents in collection: 17
